In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import re

def changes_from_press(stock_data, press_date, period):
    next_day = timedelta(days=1)
    time_after_release = timedelta(days=period)
    

    next_trading_day = press_date
    
    # try to obtain share price at open on press release date
    day_1_price = stock_data[stock_data['Date'] == press_date]['Open'].values


    # increase the date till the first available open price is found
    num_days = 1
    while not day_1_price:
        if num_days > 30:
            return None
        
        next_trading_day = next_trading_day + next_day

        day_1_price = stock_data[stock_data['Date'] == (next_trading_day)]['Open'].values
        num_days = num_days + 1

    # get next day if available
    next_trading_day = next_trading_day + time_after_release
    day_2_price = stock_data[stock_data['Date'] == (next_trading_day)]['Close'].values

    # increase the date till the second available close price is found
    num_days = 1
    while not day_2_price:
        if num_days > 30: 
            return None
        
        next_trading_day = next_trading_day + next_day

        day_2_price = stock_data[stock_data['Date'] == (next_trading_day + next_day)]['Close'].values
        num_days = num_days + 1
    
    # calcualte percent difference between share prices
    pct_change = ((day_2_price - day_1_price) / day_2_price)*100
    pct_change = pct_change[0]
    return pct_change

def get_df(ticker):
    
    stock_data = yf.Ticker(ticker)

    # get historical market data
    stock_hist = stock_data.history(period="max")

    stock_hist.reset_index(inplace=True)

    stock_hist['Date'] = pd.to_datetime(stock_hist['Date']).dt.date

    stock_hist['Pct_Close'] = stock_hist['Close'].pct_change()*100

    return stock_hist

In [4]:

year = datetime.today().year

driver = webdriver.Firefox()
driver.get('https://shareholder.ford.com/Investors/news/default.aspx')

data = []

ticker = 'jpm'

# select by value 

stock_hist = get_df(ticker)

year = datetime.today().year

while True:

    select = Select(driver.find_element(By.ID, "newsYear"))

    # select by value 
    try:
        select.select_by_value(str(year))
    except:
        break

    time.sleep(2)

    html = driver.page_source


    soup = BeautifulSoup(html) 

    articles = soup.find_all('div',attrs={'class':'module_item'})
    
    for article in articles:
        date = article.find('div', attrs={'class':'module_date-time'}).text.lstrip()
        title = article.find('div', attrs={'class':'module_headline'}).text
        
        
        date = date.lstrip().rstrip()    # remove starting and trailing whitespaces
        title = title.lstrip().rstrip() 
        title = re.sub("\s\s+", " ", title)

        date = datetime.strptime(date, '%B %d, %Y').date()
        pct_change = changes_from_press(stock_hist, date, 1)
        
        data.append([date, title, pct_change])
    print(data)
    year = year - 1

driver.close()

data = pd.DataFrame(data, columns=['date', 'press title', '1d change'])

data = data.dropna()

dates_str = [date.strftime("%m/%d/%Y") for date in data['date'].tolist()]
result = data.to_json(orient="values", index=True)


dict_data = {
    'ticker': ticker,
    'data': json.loads(result),
    },


with open(f'./data/{ticker}.json', 'w', encoding='utf-8') as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)

C:\Users\AidanLaptop\AppData\Local\Temp\ipykernel_10556\2867264320.py:43: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_2_price:


[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car

C:\Users\AidanLaptop\AppData\Local\Temp\ipykernel_10556\2867264320.py:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_1_price:


[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car

[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car

[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car

[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car

[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car

[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car

[[datetime.date(2023, 5, 3), 'Ford US April 2023 Sales Release', None], [datetime.date(2023, 5, 2), 'Ford Re-Opens Orders for Mustang Mach-E with Higher EPA Est. Range, BlueCruise 1.2 Hands-Free Tech, Lower Pricing', -2.657171669346242], [datetime.date(2023, 5, 2), 'Fords First-Quarter Volumes Sales Profits Cash Flow All Up; Distinct Segments Sharpen Focus Speed Accountability', -2.657171669346242], [datetime.date(2023, 4, 11), 'Ford’s Oakville, Ontario, Manufacturing Site Prepares to Build Next-Gen EVs; C$1.8 Billion Transformation Begins 2024', 0.1556396484375], [datetime.date(2023, 4, 6), 'Ford Motor Company Board Declares Dividend for Q2 2023', 0.6959100742012163], [datetime.date(2023, 4, 4), 'Ford Becomes America’s Best-Selling Brand in the First Quarter on Rising Sales for Trucks, Vans, Broncos, Large SUVs, EVs, Mustang', -1.468382189580985], [datetime.date(2023, 4, 3), 'Road to Better: Ford Releases 2023 Integrated Sustainability and Financial Report, Details Progress Toward Car